# Toxicity - Matteo Mistri and Daniele Papetti
**Tox21 dataset analysis and prediction**

This notebook is thought to run on colab and using google drive as storage system, so you may need to change the directories of input and output in order to make the notebook run on locally or on your drive. If you clone the repository, you will simply need to copy on your drive the notebook and the subdirectories "dumps" and "images". Moreover you need to download the datasets from source (http://bioinf.jku.at/research/DeepTox/tox21.html) and add them in the main directory of the repository; please, do note that the DENSE features and labels (of both training and test) have to be downloaded for this notebook.<br>
If you have possible suggestions to fix this problem, you can write us. Any suggestion will be appreciated :)

In order to avoid modules version problems, we highlight that the notebook was delevoped with this versions:<br>
- Keras = 2.2.5
- matplotlib = 3.1.2
- numpy = 1.17.4
- pandas = 0.25.3
- smac = 0.11.1
- tensorflow = 1.15.0

### Dataset input and module load
In this part, we load the dataset and modules which will be used in the notebook

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/'My Drive'

In [0]:
!apt-get install swig

In [0]:
pip install smac

In [0]:
pip install pyDOE

In [0]:
# suppress warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import warnings
warnings.filterwarnings("ignore")

In [0]:
# import of all required modules
import pandas as pd
import numpy as np
import pyDOE
import pickle
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from operator import itemgetter

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.model_selection import KFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight

from keras.callbacks import EarlyStopping, Callback
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization, Dropout, Activation, Input
from keras import regularizers
from keras import optimizers

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import keras.backend as kb

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace, Configuration
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, UniformIntegerHyperparameter, CategoricalHyperparameter
# Import SMAC-utilities
from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO
from smac.optimizer.acquisition import LCB
from smac.initial_design import latin_hypercube_design

import pickle

In [0]:
# load the dataset
X_train = pd.read_csv("./AML_project/tox21_dense_train.csv")
X_test = pd.read_csv("./AML_project/tox21_dense_test.csv")
Y_train = pd.read_csv("./AML_project/tox21_labels_train.csv")
Y_test = pd.read_csv("./AML_project/tox21_labels_test.csv")

### Data Preprocessing
In this section, we explore the dataset and we remove the features considered irrelevant and the instances considered outliers. 

The names of the samples are dropped becaues they are irrelevant for the classification task. 
In addition, we substitute the NaN values in the labels with the value 0 since we assume that the test was not performed by the lab researchers, because they thought that the molecule could not be involved in that biological pathway.

In [0]:
# drop first column which contains the names
X_train = X_train.drop(X_train.columns[[0]], axis = 1)
X_test = X_test.drop(X_test.columns[[0]], axis = 1)

Y_train = Y_train.drop(Y_train.columns[[0]], axis = 1)
Y_test = Y_test.drop(Y_test.columns[[0]], axis = 1)

In [0]:
# search for missing values in the features
print(X_train.isnull().any().any())
print(X_test.isnull().any().any())

In [0]:
# check if all features are numeric
set(X_train.dtypes.append(X_test.dtypes))

In [0]:
# transform NaN in 0 in the labels
print("Missing values of train labels:\n{}".format(sum(Y_train.isnull().sum(axis = 1))))
print("Missing values of test labels:\n{}".format(sum(Y_test.isnull().sum(axis = 1))))
Y_train = Y_train.fillna(0)
Y_test = Y_test.fillna(0)

We study the distribution of the labels values and the correlation among them. Moreover, we remove the examples considered as outliers and the features with zero variance, then correlation between features and labels is explored. Finally, the values of all the features are normalized.

In [0]:
# labels distribution
for l in Y_train.columns:
  unique_values_count = Counter(Y_train[l])
  print("{}: {}".format(l, unique_values_count))

  plt.figure(figsize = (8, 6))
  sns.countplot(x = l, data = Y_train)
  plt.xlabel("Values", fontsize = 16)
  plt.ylabel("Number of instances", fontsize = 16)
  plt.title(' '.join(l.split('.')), fontsize = 18)
  plt.xticks(ticks = [0, 1], labels=[0, 1], fontsize = 12)
  plt.yticks(fontsize = 12)
  plt.savefig("./AML_project/images/png/hist-{}.png".format(''.join(l.split('.'))), dpi = 300)
  plt.savefig("./AML_project/images/pdf/hist-{}.pdf".format(''.join(l.split('.'))))
  plt.close()

In [0]:
# removal of outliers
# we consider outliers what is out from 3 times the interquantile range
outliers = list()
# if we consider feature by feature, we will drop all the dataset,
# so we count how many time a record is considered outlier
for column in X_train.columns:
  mean = X_train[column].mean()
  q1 = X_train[column].quantile(1 / 4)
  q3 = X_train[column].quantile(3 / 4)
  threshold = 3 * (q3 - q1)

  # Indexes of outliers
  outliers.extend(X_train[X_train[column] > mean + threshold].index.values.tolist())      
  outliers.extend(X_train[X_train[column] < mean - threshold].index.values.tolist())

# a record is dropped if it is considered as outlier in more than a quarter of the features
out_counter = Counter(outliers)
toDrop = [k for k, v in zip(out_counter.keys(), out_counter.values()) if v > 200]
print("Removed {} outliers".format(len(toDrop)))
# removal of the outlier rows, and reset of dataframe indexes 
X_train.drop(toDrop, inplace = True)
Y_train.drop(toDrop, inplace = True)
X_train.reset_index(drop = True, inplace = True)
Y_train.reset_index(drop = True, inplace = True)

In [0]:
# remove features with zero variance since they carry no information
var_zero_columns = [c for c in X_train.columns if len(np.unique(X_train[c])) == 1]
print("Removed {} features".format(len(var_zero_columns)))
X_train = X_train.drop(var_zero_columns, axis = 1)
X_test = X_test.drop(var_zero_columns, axis = 1)

In [0]:
# correlation matrix between labels
corr_matrix = Y_train.corr().abs()

plt.figure(figsize = (8, 6))
sns.heatmap(corr_matrix, fmt = '.2f', annot = True, vmin = 0, vmax = 1,
            xticklabels = range(12), yticklabels = range(12))
plt.title("Labels correlation matrix heatmap", fontsize = 18)
plt.tight_layout()
plt.savefig("./AML_project/images/png/labels_corr_matrix_heatmap.png", dpi = 300)
plt.savefig("./AML_project/images/pdf/labels_corr_matrix_heatmap.pdf")
plt.close()

In [0]:
# correlation study between features and labels
corr_matrix = X_train.merge(Y_train, right_index = True, left_index = True).corr().abs()
# print the most correlated feature for each label
s = set()
for label in Y_train.columns:
  # extract correlation column of the considered class
  corr_col = corr_matrix[label]
  # remove the elements whose index is a label
  cleaned_col = corr_col.drop(Y_train.columns, axis = 0)
  sorted_col = cleaned_col.sort_values(ascending = False)
  # extract the most correlated features for the considered label
  s.add(sorted_col.index[0])
  print("Class {} is mosty correlated with feature {} with a value of {}".format(label, sorted_col.index[0], round(sorted_col[0], 3)))
print("Unique features: {}".format(len(s)))

In [0]:
# plot distribution of highly correlated features/labels
high_corr_features_count = {k: (sum(1 for x in corr_matrix[k] if x > 0.90) - 1) for k in corr_matrix.columns}
c = Counter(high_corr_features_count.values())

plt.Figure(figsize = (8, 6))
plt.xscale('log')
plt.xlim(left = min(c.values()) - 0.2, right = max(c.values()) + 50)
for p, v in c.items():
  plt.scatter(v, p)
plt.xlabel("# features/labels", fontsize = 16)
plt.ylabel("# highly correlated f/l", fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title("Distribution of correlated features/labels", fontsize = 18)
plt.tight_layout()
plt.savefig("./AML_project/images/png/distribution_high_corr.png", dpi = 300)
plt.savefig("./AML_project/images/pdf/distribution_high_corr.pdf")
plt.close()

In [0]:
# normalize features in 0 mean and 1 std
scaler = StandardScaler()
scaler.fit(X_train.append(X_test).values)
X_train = pd.DataFrame(scaler.transform(X_train.values), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns = X_test.columns)

### Feature reduction
In this notebook, we offer 3 possible feature reduction strategies (which are all analyzed in the associated report), in addition to the “no reduction” strategy; i.e., keeping all the remaining features of the dataset after the prepocessing step (~800).<br>
If you are interested in the use one feature reduction strategy, run the related cell; if you want to keep all the features, just skip this step.<br>
Because the SMAC module (which is used for the SMBO process) causes some problems with Keras if a different network is trained before running SMAC, after the autoencoder step the runtime must be restarted and the results produced by the encoder must be loaded. 

#### Correlation

In [0]:
# feature reduction using the correlation between the features
# if two features are highly correlated (i.e., the correlation is greater than 0.9),
# one of them is dropped
corr_matrix = X_train.corr().abs()

# select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))

# find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# drop the features
X_train = X_train.drop(X_train[to_drop], axis = 1)
X_test = X_test.drop(X_test[to_drop], axis = 1)

#### PCA

In [0]:
features_limit = 100
columns = ['col' + str(x) for x in range(features_limit)]
PCA_transformer = PCA(n_components = features_limit)
PCA_transformer.fit(X_train.values)
X_train = pd.DataFrame(PCA_transformer.transform(X_train.values), columns = columns)
X_test = pd.DataFrame(PCA_transformer.transform(X_test.values), columns = columns)

#### Autoencoder

In [0]:
# Define early stopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', 
                   verbose = 1, patience = 15, min_delta = 0.001,
                   restore_best_weights = True)

encoding_dim1 = int(X_train.shape[1] / 2)
encoding_dim2 = int(encoding_dim1 / 2)
encoding_dim3 = 100
columns = ['col' + str(x) for x in range(encoding_dim3)]

input_layer = Input(shape = (X_train.shape[1],))
# "encoded" is the encoded representation of the input
encoded1 = Dense(encoding_dim1, activation = 'relu')(input_layer)
encoded2 = Dense(encoding_dim2, activation = 'relu')(encoded1)
bottleneck = Dense(encoding_dim3, activation = 'relu')(encoded2)
# "decoded" is the lossy reconstruction of the input
decoded2 = Dense(encoding_dim2, activation = 'relu')(bottleneck)
decoded1 = Dense(encoding_dim1, activation = 'relu')(decoded2)
output_layer = Dense(X_train.shape[1], activation = 'linear')(decoded1)
# this model maps an input to its reconstruction
autoencoder = Model(input_layer, output_layer)
# this model maps an input to its encoded representation
encoder = Model(input_layer, bottleneck)

# compile the model
autoencoder.compile(optimizer = 'adam', loss = 'mse')
autoencoder.summary()
# fit the autoencoder
autoencoder.fit(X_train.values, X_train.values, 
                validation_split = 0.1, epochs = 300, batch_size = 256, 
                verbose = True, callbacks = [es], use_multiprocessing = True)
# extract representation
X_train = pd.DataFrame(encoder.predict(X_train.values), columns = columns)
X_test = pd.DataFrame(encoder.predict(X_test.values), columns = columns)

# dumping the results in order to make the loadable due to issues with smac
f = open("./AML_project/dumps/X_train_autoencoder.pkl","wb")
pickle.dump(X_train,f)
f.close()
f = open("./AML_project/dumps/X_test_autoencoder.pkl","wb")
pickle.dump(X_test,f)
f.close()

If you used the autoencoder FE approach, you need to restart the runtime and run the following cell, since SMAC get stuck if we train any other model before it.

In [0]:
infile = open("./AML_project/dumps/X_train_autoencoder.pkl",'rb')
X_train = pickle.load(infile)
infile.close()
infile = open("./AML_project/dumps/X_test_autoencoder.pkl",'rb')
X_test = pickle.load(infile)
infile.close()

### Testing of some topologies
In this part, we define some functions which will be used in the remaining part of the notebook. We also show the performance of three different topologies with different depth. All the NN have hidden layers with depth starting from 512, the depth of a layer is the half of the previous one. So, we test netwrks with 3,4 and 5 hidden layers. Additional tests with different netowrks were performend in a more embrional state of this work, so they results are not present anymore, but they did not fluctuate significantly from the ones presented.<br>
Finally, all the networks have 12 output neurons with a sigmoid activation function: each of them represent a predicted value for a signle label, remember that is not a one-hot encoding, because a molecule can be positive to different toxicological experiments.

In [0]:
# evaluate auc for a given model
# it returns a dictionary with the auc for each label and the mean of the aucs
def evaluate_performance(model, test_features, test_label):
  test_predictions = model.predict(test_features)
  test_pred_df = pd.DataFrame(data = test_predictions, columns = test_label.columns)
  auc = dict()
  for c_pred, c_true in zip(test_pred_df, test_label):
    auc[c_true] = roc_auc_score(y_true = test_label[c_true], y_score = test_pred_df[c_pred])

  return((auc, np.mean(list(auc.values()))))

Technically speaking, this is a multi-label task, and keras do not offer any loss for these particular kind of tasks. So, we defined a custom binary crossentropy; to do that, we searched and adapted a solution found online.

In [0]:
# def custom loss: weighted_binary_crossentropy
def weighted_binary_crossentropy(POS_WEIGHT):
  def loss(target, output):
    # transform back to logits
    _epsilon = kb.tensorflow_backend._to_tensor(kb.tensorflow_backend.epsilon(), 
                                                output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets = target,
                                                    logits = output,
                                                    pos_weight = POS_WEIGHT)
    return tf.reduce_mean(loss, axis = -1)
  return loss

The output of the following cell is the performance of the different netowrks presented previously. 

In [0]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, 
                    patience = 10, min_delta = 0.001, restore_best_weights = True)

kfold = KFold(n_splits = 3, shuffle = True)
POS_WEIGHT = 20

for t, v in kfold.split(X_train, Y_train):
  aucs = list()
  for i in range(3):
    # NN
    NN = Sequential()
    NN.add(Dense(512, input_shape = (X_train.shape[1],)))
    NN.add(BatchNormalization())
    NN.add(Activation('relu'))
    NN.add(Dropout(rate = 0.2))
    NN.add(Dense(256))
    NN.add(BatchNormalization())
    NN.add(Activation('relu'))
    NN.add(Dropout(rate = 0.2))
    NN.add(Dense(128))
    NN.add(BatchNormalization())
    NN.add(Activation('relu'))
    NN.add(Dropout(rate = 0.2))
    if i >= 1:
      NN.add(Dense(64))
      NN.add(BatchNormalization())
      NN.add(Activation('relu'))
      NN.add(Dropout(rate = 0.2))
    if i >= 2:
      NN.add(Dense(32))
      NN.add(BatchNormalization())
      NN.add(Activation('relu'))
      NN.add(Dropout(rate = 0.2))
    NN.add(Dense(12, activation = 'sigmoid'))

    # Compile model
    NN.compile(optimizer = 'adam', 
              loss = weighted_binary_crossentropy(POS_WEIGHT = POS_WEIGHT))
    # Fit tne network
    learning_process_NN = NN.fit(X_train.iloc[t], Y_train.iloc[t], 
                                validation_data = (X_train.iloc[v], Y_train.iloc[v]), 
                                callbacks = [es], epochs = 300, batch_size = 128, 
                                verbose = False, use_multiprocessing = True)
    aucs.append(evaluate_performance(NN, X_train.iloc[v], Y_train.iloc[v])[1])

  print("The average AUC in the 3 fold CV is {}".format(np.mean(aucs)))

Since the deeper topologies do not bring any advantage, we decide to use a simpler topology (Occam's razor): three hidden layers with a depth of 512, 256 and 128 respectively.

### Hyperparameter optimization
In the following, we apply the task of hyperparameter optimization. We decided to do that after the preliminary exploration of possible topologies, mainly because it is time consuming. The budget used is of 120 possible points in the search space.

In [0]:
# define early stopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, 
                    patience = 10, min_delta = 0.001, restore_best_weights = True)

kfold = KFold(n_splits = 3, shuffle = True)
# function that returns 1 - mean of average auc over 3 folds
def create_compute_model(config):
  POS_WEIGHT = config['pos_weight']
  BATCH_SIZE = config['batch_size']
  DROPOUT_RATE = config['dropout_rate']
  ACTIVATION = config['activation']
  L2_REG = config['l2_reg']

  aucs = list()

  for t, v in kfold.split(X_train, Y_train):
    # NN
    NN = Sequential()
    NN.add(Dense(512, input_shape = (X_train.shape[1],), 
                 kernel_regularizer = regularizers.l2(L2_REG)))
    NN.add(BatchNormalization())
    NN.add(Activation(ACTIVATION))
    NN.add(Dropout(rate = DROPOUT_RATE))
    NN.add(Dense(256, kernel_regularizer = regularizers.l2(L2_REG)))
    NN.add(BatchNormalization())
    NN.add(Activation(ACTIVATION))
    NN.add(Dropout(rate = DROPOUT_RATE))
    NN.add(Dense(128, kernel_regularizer = regularizers.l2(L2_REG)))
    NN.add(BatchNormalization())
    NN.add(Activation(ACTIVATION))
    NN.add(Dropout(rate = DROPOUT_RATE))
    NN.add(Dense(12, activation = 'sigmoid'))

    # compile model
    NN.compile(optimizer = 'adam', 
               loss = weighted_binary_crossentropy(POS_WEIGHT = POS_WEIGHT))
    # fit tne network
    learning_process_NN = NN.fit(X_train.iloc[t], Y_train.iloc[t], 
                                 validation_data = (X_train.iloc[v], Y_train.iloc[v]), 
                                 callbacks = [es], epochs = 300, batch_size = BATCH_SIZE, 
                                 verbose = False, use_multiprocessing = True)
    aucs.append(evaluate_performance(NN, X_train.iloc[v], Y_train.iloc[v])[1])
    
    # clear model representation
    NN = None
    learning_process_NN = None
  print("The average AUC in the 3 fold CV is {}".format(np.mean(aucs)))
  return 1 - np.mean(aucs)

The search space is defined with the usage of these hyperparameter:
- pos_weight is a parameter for the loss function, we treat it as a continuous hyperparameter which can vary from 0.1 to 50;
- batch_size is the dimension of the batch during the trainig of the model, it is categorical and it can assume the following values: 64, 128, 256, 512 and 1024; they are all potence of 2 because of efficency with GPUs;
- dropout_rate is the probability of the neurons to be turned off during the evaluation of a single batch; it is treated as continuous and it can vary from 0 (i.e., no dropout) from 0.7;
- l2_reg is the regularization hyperparameter, it is treated as continuou and it spans form 0 to 0.2;
- activation is the possible activation function used by the hidden layers;

In [0]:
# define search spaces
cs = ConfigurationSpace()
pos_weight = UniformFloatHyperparameter('pos_weight', 0.1, 50, default_value = 10)
batch_size = CategoricalHyperparameter('batch_size', [2 ** i for i in range(6, 11)], 
                                       default_value = 128)
dropout_rate = UniformFloatHyperparameter('dropout_rate', 0.0, 0.7, default_value = 0.2)
l2_reg = UniformFloatHyperparameter('l2_reg', 0.0, 0.2, default_value = 0.001)
activation = CategoricalHyperparameter('activation', ['relu', 'tanh', 'exponential', 'linear', 'selu'], 
                                       default_value = 'relu')

cs.add_hyperparameters([pos_weight, batch_size, dropout_rate, l2_reg, activation])

In [0]:
# define scenario
scenario = Scenario({"run_obj": "quality",
                     "runcount-limit": 120,
                     "cs": cs,
                     "deterministic": "True"
                     })

In [0]:
s_smac = SMAC4HPO(scenario = scenario, tae_runner = create_compute_model, 
                     acquisition_function = LCB, 
                     initial_design = latin_hypercube_design.LHDesign,
                     initial_design_kwargs = {'max_config_fracs': 1 / 6,
                                              'n_configs_x_params': 3})

s_incumbent = s_smac.optimize()

s_inc_value = create_compute_model(s_incumbent)

print("Optimized Value: %.4f" % (1 - s_inc_value))

In [0]:
# collect the results of the optimization and dump them in order to make them available 
# for further analysis without running the process again 
cfg_acc = dict()
bs = dict()

cfg_acc['no_fs'] = list()
bs['no_fs'] = list()
best_seen = 1 - s_smac.get_runhistory().get_cost(s_smac.get_runhistory().get_all_configs()[0])
for config in s_smac.get_runhistory().get_all_configs():
  loss = s_smac.get_runhistory().get_cost(config)
  auc = 1 - loss
  cfg_acc['no_fs'].append((config, auc))
  if auc > best_seen:
    best_seen = auc
  bs['no_fs'].append(best_seen)

print(bs['no_fs'])
 
f = open("./AML_project/dumps/no_fs_cfg_acc.pkl","wb")
pickle.dump(cfg_acc,f)
f.close()
f = open("./AML_project/dumps/no_fs_bs.pkl","wb")
pickle.dump(bs,f)
f.close()

To collect data for each feature reduction approach (and no feature reduction) we executed the hyperparameter optimization (the code above) 4 different times and dumped the results each time in a different file.

### Analisys of hyperparameter optimization process results
In this section, we analyze the results obtained by the optimization process: the best-seen configuration and, in some sense, if and how the search space was explored.
First of all, we load the results of each optimization task (one optimization for each possible input strategy). Then, we plot the data to analyze what we said below, and finally we print the performance obtained by the network with the optimal hyperparameter for each input strategy.

In [0]:
# Gathering the results together
bs = dict()
cfg_auc = dict()
possible_inputs = ["no_fs", "corr", "pca", "autoencoder"]
for x in possible_inputs:
  infile = open("./AML_project/dumps/{}_cfg_auc.pkl".format(x),'rb')
  cfg_auc.update(pickle.load(infile))
  infile.close()
  infile = open("./AML_project/dumps/{}_bs.pkl".format(x),'rb')
  bs.update(pickle.load(infile))
  infile.close()

labels = {"no_fs": "No reduction", "corr": "Correlation", "pca": "PCA", "autoencoder": "Autoencoder"}

In [0]:
x = list(range(120))
plt.figure(figsize = (8, 6))
colors = ["red", "orange", "blue", "black"]
for k, c in zip(bs.keys(), colors):
  plt.plot(x, bs[k], '-', color = c, label = labels[k], linewidth = 1.5)

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title("Best seen", fontsize = 18)
plt.xlabel("Iteration", fontsize = 16)
plt.ylabel("Mean of AUC", fontsize = 16)
plt.legend(loc = 'lower right')
plt.savefig("./AML_project/images/pdf/best-seen.pdf")
plt.savefig("./AML_project/images/png/best-seen.png", dpi = 300)
plt.close()

In [0]:
plt.figure(figsize = (8, 6))
for k, c in zip(cfg_auc.keys(), colors):
  Y = [e[1] for e in cfg_auc[k]]
  plt.plot(x, Y, color = c, label = labels[k], linewidth = 1.5)

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title("AUC of configuration evaluation", fontsize = 18)
plt.xlabel("Configuration", fontsize = 16)
plt.ylabel("Mean of AUC", fontsize = 16)
plt.legend(loc = 'lower right')
plt.savefig("./AML_project/images/pdf/AUC-iteration.pdf")
plt.savefig("./AML_project/images/png/AUC-iteration.png", dpi = 300)
plt.close()

In [0]:
# print best config for each method
from operator import itemgetter
for method in ["no_fs", "corr", "pca", "autoencoder"]:
  tmp = max(cfg_auc[method], key = itemgetter(1))
  print("Best configuration for {} method was \n{}\n with a mean AUC value of {}\n\n".format(method, tmp[0], tmp[1]))

### Analysis of the best model
In this final part, we study deeply the behaviour of the neural network with the optimal configuration of the hyperparameters. First of all, we define some functions which will be used. Then, we train the network monitoring the loss and the mean of the auc. Finally, we plot how our model perform in comparison of what presented during the challenge and we study how the threshold of the %TP (Ture Positive percentage) affects the recall and precision of the model.

In [0]:
# plot the evolution of loss function during the epochs of the training of the network, for
# both train and validation set
def plot_history(network_history):
  plt.figure()
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.axvline(x = len(network_history.history['loss']) - 10, linestyle= '--', color = 'red', linewidth = 1.5)
  plt.plot(network_history.history['loss'])
  plt.plot(network_history.history['val_loss'])
  plt.legend(['Training', 'Validation'])

  plt.savefig("./AML_project/images/pdf/learning_process.pdf")
  plt.savefig("./AML_project/images/png/learning_process.png", dpi = 300)
  plt.close()

In [0]:
# class to evaluate auc and roc at each step of the network training
class roc_callback(Callback):
  def __init__(self,training_data,validation_data):
    self.x = training_data[0]
    self.y = training_data[1]
    self.x_val = validation_data[0]
    self.y_val = validation_data[1]

  def on_train_begin(self, logs = dict()):
    return

  def on_train_end(self, logs = dict()):
    return

  def on_epoch_begin(self, epoch, logs = dict()):
    return

  def on_epoch_end(self, epoch, logs = dict()):
    y_pred = pd.DataFrame(data = self.model.predict(self.x), columns = self.y.columns)
    y_pred_val = pd.DataFrame(data = self.model.predict(self.x_val), columns = self.y.columns)
    average_roc = 0.0
    average_roc_val = 0.0
    for i in self.y.columns:
      average_roc += roc_auc_score(self.y[i], y_pred[i])
      average_roc_val += roc_auc_score(self.y_val[i], y_pred_val[i])

    average_roc = average_roc / 12
    average_roc_val = average_roc_val / 12
    print('Average-roc-auc: {}   Average-roc-auc_val: {}'.format(round(average_roc, 4), 
                                                                  round(average_roc_val,4)))
    return

    def on_batch_begin(self, batch, logs = dict()):
      return

    def on_batch_end(self, batch, logs = dict()):
      return

In [0]:
# confusion matrix as a dictionary, very naive implementation
# predict and evaluate performances gathering them in the confusion matrix
def get_confusion_matrix(model, tpr_threshold, test_features, test_labels):
  thresholds = get_thresholds(tpr_threshold, model, test_features, test_labels)
  predictions = model.predict(test_features)
  d = {k: {'t1': 0, 't0': 0, 'f1': 0, 'f0': 0} for k in test_labels.columns}
  for preds, trues in zip(predictions, test_labels.itertuples()):
    for p, t, k, i in zip(preds, trues[1:], test_labels.columns, range(12)):
      p = 1 if p > thresholds[i] else 0
      if p == t and p == 0:
          d[k]['t0'] = d[k]['t0'] + 1
      if p == t and p == 1:
          d[k]['t1'] = d[k]['t1'] + 1
      if p != t and p == 0:
          d[k]['f0'] = d[k]['f0'] + 1
      if p != t and p == 1:
          d[k]['f1'] = d[k]['f1'] + 1
  return d, thresholds

In [0]:
# function which prints the confusion matrix
def print_confusion_matrix(confusion_matrix, label):
  predicted_header = ''.join([' '] * 12) + "PREDICTED" + ' '
  class_header = ''.join([' '] * 14) + "CLASS" + ''.join([' '] * 3)
  class_values = ''.join([' '] * 13) + "1" + ''.join([' '] * 5) + "0" + ''.join([' '] * 2)
  header = '\n'.join([predicted_header, class_header, class_values])
  first_row = "ACTUAL  1" + ''.join([' '] * 2) + '{0:^5d}'.format(confusion_matrix[label]['t1']) + ' ' + '{0:^5d}'.format(confusion_matrix[label]['f0'])
  second_row = " CLASS  0" + ''.join([' '] * 2) + '{0:^5d}'.format(confusion_matrix[label]['f1']) + ' ' + '{0:^5d}'.format(confusion_matrix[label]['t0'])
  to_print = '\n'.join([header, first_row, second_row]) + '\n'
  print(to_print)

In [0]:
# given the roc curve, calculate the threshold to obtain the %tp (tpr_threshold)
def get_thresholds(tpr_threshold, model, X, Y):
  test_predictions = model.predict(X)
  test_pred_df = pd.DataFrame(data = test_predictions, columns = Y.columns)
  prediction_thresholds = []
  for i in range (12):
    # note that tpr are already sorted ascending
    _, tpr, thresholds = roc_curve(Y.iloc[:, i], test_pred_df.iloc[:, i])
    for tpr_it, thresh_it in zip (tpr, thresholds):
      if tpr_it >= tpr_threshold:
        prediction_thresholds.append(thresh_it)
        break
  return prediction_thresholds

In [0]:
# train with optimal hyperparameters
POS_WEIGHT = 0.8226761093444144
BATCH_SIZE = 256
DROPOUT_RATE = 0.3573514213523089
ACTIVATION = 'relu'
L2_REG = 0.0001080355932884863

X_t, X_v, Y_t, Y_v = train_test_split(X_train, Y_train, test_size = 0.20)

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, 
                    patience = 10, min_delta = 0.001, restore_best_weights = True)

NN = Sequential()
NN.add(Dense(512, input_shape = (X_train.shape[1],), 
              kernel_regularizer = regularizers.l2(L2_REG)))
NN.add(BatchNormalization())
NN.add(Activation(ACTIVATION))
NN.add(Dropout(rate = DROPOUT_RATE))
NN.add(Dense(256, kernel_regularizer = regularizers.l2(L2_REG)))
NN.add(BatchNormalization())
NN.add(Activation(ACTIVATION))
NN.add(Dropout(rate = DROPOUT_RATE))
NN.add(Dense(128, kernel_regularizer = regularizers.l2(L2_REG)))
NN.add(BatchNormalization())
NN.add(Activation(ACTIVATION))
NN.add(Dropout(rate = DROPOUT_RATE))
NN.add(Dense(12, activation = 'sigmoid'))

# Compile model
NN.compile(optimizer = 'adam', 
            loss = weighted_binary_crossentropy(POS_WEIGHT = POS_WEIGHT))
# Fit tne network
learning_process_NN = NN.fit(X_t, Y_t, 
                              validation_data = (X_v, Y_v), 
                              callbacks = [roc_callback(training_data = (X_t, Y_t), 
                              validation_data = (X_v, Y_v)), es],
                              epochs = 300, batch_size = BATCH_SIZE, 
                              verbose = True, use_multiprocessing = True)
plot_history(learning_process_NN)

In [0]:
# print ROC curves for all classes
test_predictions = NN.predict(X_test)

test_pred_df = pd.DataFrame(data = test_predictions, columns = Y_test.columns)
plt.figure(figsize = (8, 6))
for i in range(12):
  fpr, tpr, threshold = roc_curve(Y_test.iloc[:, i], test_pred_df.iloc[:, i])
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, label = ' '.join(Y_test.columns[i].split('.')))

plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('Receiver Operating Characteristic', fontsize = 18)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate', fontsize = 16)
plt.xlabel('False Positive Rate', fontsize = 16)
plt.tight_layout()

plt.savefig("./AML_project/images/pdf/ROC.pdf")
plt.savefig("./AML_project/images/png/ROC.png", dpi = 300)
plt.close()

In [0]:
# plot how our model performs in comparison with all other models that partecipated to the
# challenge and reported in the DeepTox paper. Their performance are stored in a csv, where 0
# stands for a NaN value
our_results, auc_mean = evaluate_performance(NN, X_test, Y_test)
our_results = {k.replace('.', '_'): v for k, v in our_results.items()}
perf_df = pd.read_csv("./AML_project/paper_perf.csv")
our_results['AVG'] = auc_mean
nr_keys = ["NR_AhR", "NR_AR", "NR_AR_LBD", "NR_Aromatase", "NR_ER", "NR_ER_LBD", "NR_PPAR_gamma"]
sr_keys = ["SR_ARE", "SR_ATAD5", "SR_HSE", "SR_MMP", "SR_p53"]
our_results['NR'] = np.mean([our_results[k] for k in nr_keys])
our_results['SR'] = np.mean([our_results[k] for k in sr_keys])
our_results['model'] = "our"
perf_df = perf_df.append(our_results, ignore_index = True)

colors = 'red-orangered-darkorange-orange-gold-forestgreen-darkgreen-green-lightseagreen-cyan-deepskyblue-navy-blue-indigo-darkviolet-purple-crimson-saddlebrown-black'.split('-')
label_map = {l: i for l, i in zip(perf_df.columns, range(len(perf_df.columns))) if l != 'model'}
colors_map = {m: x for m, x in zip(perf_df['model'], colors)}
legend_plot_best2 = dict() # the bold and gray background in the table of the paper

plt.figure(figsize = (8, 6), dpi = 300)
for r in perf_df.itertuples(index = False):
  r = r._asdict()
  model = r.pop('model') # r is  also updated
  artist = None
  for k in r:
    if r[k] == 0: # in the csv, 0 means empty value
      continue
    artist, = plt.plot(r[k], label_map[k], color = colors_map[model], 
                       alpha = 1 if model == 'our' else 0.9,
                       marker = 'x' if model == 'our' else '|',
                       markersize = 12 if model == 'our' else 10,
                       markeredgewidth = 2 if model == 'our' else 2,
                       linestyle = 'None')
  if model in ['DeepTox21', 'AMAZIZ', 'dmlab', 'microsomes', 'NCI', 'our']:
    legend_plot_best2[model] = artist

plt.xlim(left = 0.26, right = 1)
xticks = [0.3, 0.4, 0.5, 0.6, 0.7]
xticks.extend([round(0.7 + x * 0.05, 2) for x in range(1, 7)])
plt.xticks(ticks = xticks, labels = xticks, fontsize = 14, rotation = 45)

yticks = list()
for l in label_map.keys():
  if l == "NR_PPAR_gamma":
    yticks.append("NR PPAR \u03B3")
    continue
  if l == "NR_Aromatase":
    yticks.append("NR Arom.")
    continue
  yticks.append(l.replace('_', ' '))

plt.yticks(ticks = [v for v in label_map.values()], 
           labels = yticks, 
           fontsize = 14)
plt.axhline(y = 3.5, color = 'r', linestyle = '-', linewidth = 1)
plt.title("Comparison between different classifiers", fontsize = 18)
plt.xlabel("AUC", fontsize = 16)
plt.ylabel("Model", fontsize = 16)
plt.legend([a for a in legend_plot_best2.values()], 
           [x for x in legend_plot_best2.keys()],
           numpoints = 1)
plt.tight_layout()
plt.savefig("./AML_project/images/pdf/comparison.pdf")
plt.savefig("./AML_project/images/png/comparison.png")
plt.close()

In [0]:
# function that returns a dictionary label: perf, where perf is a dictionary with precision
# and recall of the label. Those metrics are computed on the confusion matrix.
def precision_recall_from_cm(cm):
  each_class_pr = {k: dict() for k in cm.keys()}
  for k in cm:
    each_class_pr[k]['precision'] = cm[k]['t1'] / (cm[k]['t1'] + cm[k]['f1'])
    each_class_pr[k]['recall'] = cm[k]['t1'] / (cm[k]['t1'] + cm[k]['f0'])
  return each_class_pr

In [0]:
# plot the evolution of precision and recall varying the threshold of TruePositive rate
ts = np.linspace(0.1, 1, 10, endpoint = True)
precisions_m = list()
recalls_m = list()
precisions_std = list()
recalls_std = list()
for t in ts:
  cm, _ = get_confusion_matrix(NN, t, X_test, Y_test)
  p_r = precision_recall_from_cm(cm)
  ps = [p_r[k]['precision'] for k in p_r]
  precisions_m.append(np.mean(ps))
  precisions_std.append(np.std(ps))
  rs = [p_r[k]['recall'] for k in p_r]
  recalls_m.append(np.mean(rs))
  recalls_std.append(np.std(rs))

plt.figure(figsize = (8, 6), dpi = 300)

plt.plot(ts, [m - s for m, s in zip(precisions_m, precisions_std)],
			 color = 'green', linestyle = '-', linewidth = 0.2, antialiased = True,
			 label = "Average precision std")
plt.plot(ts, [m + s for m, s in zip(precisions_m, precisions_std)],
			 color = 'green', linestyle = '-', linewidth = 0.2, antialiased = True)

plt.fill_between(ts, [m - s for m, s in zip(precisions_m, precisions_std)],
                 [m + s for m, s in zip(precisions_m, precisions_std)],
                 color = 'green', alpha = 0.5)

plt.plot(ts, [m - s for m, s in zip(recalls_m, recalls_std)],
			 color = 'darkturquoise', linestyle = '-', linewidth = 0.2, antialiased = True,
			 label = "Average precision std")
plt.plot(ts, [m + s for m, s in zip(recalls_m, recalls_std)],
			 color = 'darkturquoise', linestyle = '-', linewidth = 0.2, antialiased = True)

plt.fill_between(ts, [m - s for m, s in zip(recalls_m, recalls_std)],
                 [m + s for m, s in zip(recalls_m, recalls_std)],
                 color = 'darkturquoise', alpha = 0.5)

plt.plot(ts, precisions_m, '-', color = 'green', linewidth = 2.5,
         label = "Precision", antialiased = True)
plt.plot(ts, recalls_m, '-', color = 'darkturquoise', linewidth = 2.5, 
         label = "Recall", antialiased = True)

plt.xlim(left = 0.0995, right = 1.0005)
plt.ylim(bottom = 0.0, top = 1.0)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlabel("%TP", fontsize = 16)
plt.ylabel("Value", fontsize = 16)
plt.title("Evolution of Precision and Recall", fontsize = 18)
plt.legend()
plt.tight_layout()
plt.savefig("./AML_project/images/pdf/pr_evolution.pdf")
plt.savefig("./AML_project/images/png/pr_evolution.png")
plt.close()